In [1]:
import sqlite3

In [3]:
connection = sqlite3.connect('mydb.db')

In [4]:
connection

In [10]:
table_creation_query = """
CREATE TABLE IF NOT EXISTS employees(
    emp_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    hire_date TEXT NOT NULL,
    salary REAL NOT NULL
);
"""

In [11]:
table_creation_query2 = """
CREATE TABLE IF NOT EXISTS customers(
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    phone TEXT
);
"""

In [12]:
table_creation_query3 = """
CREATE TABLE IF NOT EXISTS orders(
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER NOT NULL,
    order_date TEXT NOT NULL,
    amount REAL NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers (customer_id)
);
"""

In [13]:
cursor = connection.cursor()

In [14]:
cursor.execute(table_creation_query)
cursor.execute(table_creation_query2)
cursor.execute(table_creation_query3)

In [15]:
insert_query_employees = """
INSERT INTO employees (emp_id, first_name, last_name, email, hire_date, salary)
VALUES (?, ?, ?, ?, ?, ?)
"""

insert_query_customers = """
INSERT INTO customers (customer_id, first_name, last_name, email, phone)
VALUES (?, ?, ?, ?, ?)
"""


insert_query_orders = """
INSERT INTO orders (order_id, customer_id, order_date, amount)
VALUES (?, ?, ?, ?)
"""

In [16]:
employee_data = [
    (1, "employee1", "last1", "employee1@gmail.com", "2021-01-01", 10000.00),
    (2, "employee2", "last2", "employee2@gmail.com", "2022-02-01", 20000.00),
    (3, "employee3", "last3", "employee3@gmail.com", "2023-03-01", 30000.00),
    (4, "employee4", "last4", "employee4@gmail.com", "2024-04-01", 40000.00),
]


customers_data = [
    (1, "king1", "last1", "king1@gmail.com", "112345678"),
    (2, "king2", "last2", "king2@gmail.com", "122345678"),
    (3, "king3", "last3", "king3@gmail.com", "123345678"),
    (4, "king4", "last4", "king4@gmail.com", "123445678"),
    
]


orders_data = [
    (1, 1, "2023-12-01", 250.00),
    (2, 2, "2023-23-01", 150.00),
    (3, 3, "2023-11-01", 300.00),
    (4, 4, "2023-04-03", 450.00),
]

In [17]:
cursor.executemany(insert_query_employees, employee_data)
cursor.executemany(insert_query_customers, customers_data)
cursor.executemany(insert_query_orders, orders_data)

In [18]:
connection.commit()

In [19]:
cursor.execute("select * from orders")

In [20]:
for row in cursor.fetchall():
    print(row)

(1, 1, '2023-12-01', 250.0)
(2, 2, '2023-23-01', 150.0)
(3, 3, '2023-11-01', 300.0)
(4, 4, '2023-04-03', 450.0)
